In [ ]:
from nltk.translate.bleu_score import corpus_bleu
import re
import string
from random import uniform

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
import os
from google.colab import drive 
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/NLP/Challenge_5')

Mounted at /content/drive


In [ ]:
source_raw = open("train-source.txt", "r").read().split("\n")
target_raw = open("train-target.txt", "r").read().split("\n")
source_raw.pop()   #to make last characater </s>
target_raw.pop()

''

In [ ]:
punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~`…”“'''
source = []
target = []
for w in source_raw:
    if not w in punc:
        source.append(w)
for w in target_raw:
    if not w in punc:
        target.append(w)

In [ ]:
print(len(source))
print(len(target))
print(source[0])
print(target[0])
print(source[len(source)-1])
print(target[len(target)-1])

804680
804198
<s>
<s>
</s>
</s>


In [ ]:
source_vocab = set(source)
target_vocab = set(target)
source_vocab.remove("<s>")
target_vocab.remove("<s>")
source_vocab.remove("</s>")
target_vocab.remove("</s>")
print(len(source_vocab))
print(len(target_vocab))
print(len(source_vocab)*len(target_vocab))

34582
29098
1006267036


In [ ]:
source_sent = []
target_sent = []
sentence = []
for token in source:
    if token == "</s>":
        source_sent.append(sentence)
        sentence = []
    elif token != "<s>":
        sentence.append(token)
for token in target:
    if token == "</s>":
        target_sent.append(sentence)
        sentence = []
    elif token != "<s>":
        sentence.append(token)

In [ ]:
print(len(source_sent))
print(len(target_sent))

45171
45171


In [ ]:
tot = 0
for i in range(len(source_sent)):
    if len(source_sent[i])==len(target_sent[i]):
        tot = tot + 1
        
print(tot/len(source_sent))

0.6842000398485754


In [ ]:
alignments = dict()
for s in source_vocab:
    alignments[s] = dict()   

In [ ]:
source_chunks = []
target_chunks = []
for i in range(len(source_sent)):   #First pass for freebies and chunk generation. Separating by "anchor" words didn't work as well as I thought, so instead the chunks are just all words that don't have an identi
    s = source_sent[i]
    t = target_sent[i]
    if len(s) == len(t):
        for j in range(len(s)):
            alignments[s[j]][t[j]] = alignments[s[j]].get(t[j],0) + 1
    else:
        chunk_s = []
        chunk_t = []
        for w in s:
            if w in t:
                alignments[w][w] = alignments[w].get(w,0)
            else:
                chunk_s.append(w)
        for w in t:
            if not w in s:
                chunk_t.append(w)
        source_chunks.append(chunk_s)
        target_chunks.append(chunk_t)

In [ ]:
print(len(source_chunks))
print(len(target_chunks))

14265
14265


In [ ]:
for i in range(len(source_chunks)):   #second pass to lazily process chunks
    for a in source_chunks[i]:
        best_word = ""
        best_count = -1
        for b in target_chunks[i]:
            if b in alignments[a]:
                if alignments[a][b] > best_count:
                    best_count = alignments[a][b]
                    best_word = b
        if best_count > -1:
            alignments[a][best_word] = alignments[a].get(best_word) + 1

Now for the neural implementation.

In [ ]:
batch_size = 64  # Batch size for training.
epochs = 1  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.

In [ ]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

def squish(a):
  sent = ""
  for i in range(len(a)):
    if a[i] != '<s>' and a[i] != '</s>':
      if i > 0 and i < len(a)-1:
        sent = sent + " "
      sent = sent + (a[i])
  return sent

for x in source_sent:
  s = squish(x)
  for c in s:
    if c not in input_characters:
      input_characters.add(c)
  input_texts.append(s)
for y in target_sent:
  t = squish(y)
  for c in t:
    if not c in target_characters:
      target_characters.add(c)
  target_texts.append("\t" + t + "\n")
  #target_texts.append(t)

In [ ]:
input_texts = input_texts[:num_samples]
target_texts = target_texts[:num_samples]

In [ ]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

Number of samples: 10000
Number of unique input tokens: 90
Number of unique output tokens: 79
Max sequence length for inputs: 455
Max sequence length for outputs: 430


In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        if char == '\t' or char == '\n':
          continue
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [ ]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [ ]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
# Save model
model.save("s2s")


125/125 [==============================] - 519s 4s/step - loss: 0.8317 - accuracy: 0.8374 - val_loss: 0.5454 - val_accuracy: 0.8512


INFO:tensorflow:Assets written to: s2s/assets


INFO:tensorflow:Assets written to: s2s/assets


In [ ]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    #target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [ ]:
# Baseline leave sentence unchanged; BLEU score is still > 0.5!
def dumb_translation(source_sent):
  return source_sent

In [ ]:
def from_scratch_translation(source_sent):   #greedy decoding
    translation = []
    for s in source_sent:
        if s in alignments and bool(alignments[s]):
            translation.append(max(alignments[s], key=alignments[s].get))
        else:
            translation.append(s)
    return translation

In [ ]:
# Slightly better baseline is to apply a few global spelling changes
def anything_goes_translation(source_sent):
  sent = squish(source_sent)
  s = ""
  for c in sent:
    if not c in punc:
      s = s + c
  v = np.zeros(
    (1, max_encoder_seq_length, num_encoder_tokens), dtype="float32"
  )
  for t, char in enumerate(s):
    v[0, t, input_token_index[char]] = 1.0
  v[0, t + 1 :, input_token_index[" "]] = 1.0
  return decode_sequence(v[0:1])

In [ ]:
anything_goes_translation(["Cinnte"])

'Bh                                                                                                                                                                                                                                                                                                                                                                                                                                             '

In [ ]:
def evaluate():
  testsource = open('test-source.txt', 'r')
  sentence = []
  dumb_hypotheses = []
  from_scratch_hypotheses = []
  anything_goes_hypotheses = []
  for line in testsource:
    token = line.rstrip("\n")
    if token == '<s>':
      sentence = []
    elif token == '</s>':
      dumb_hypotheses.append(dumb_translation(sentence))
      from_scratch_hypotheses.append(from_scratch_translation(sentence))
      anything_goes_hypotheses.append(anything_goes_translation(sentence))
    else:
      sentence.append(token)
  references = []
  testtarget = open('test-target.txt', 'r')
  for line in testtarget:
    token = line.rstrip("\n")
    if token == '<s>':
      sentence = []
    elif token == '</s>':
      references.append([sentence])
    else:
      sentence.append(token)
  return (corpus_bleu(references,dumb_hypotheses), corpus_bleu(references,from_scratch_hypotheses), corpus_bleu(references,anything_goes_hypotheses))

In [ ]:
evaluate()